In [1]:
!pip install surprise

Genome tags:

1058: Unintentionally funny 
933: So bad its funny
934: So bad its good
891: Scifi cult
616: Low budget
274: Cult
275: Cult classic
276: Cult film
198: Cheesy


genome scores
genome tags
tags

In [2]:
import pandas as pd
import numpy as np

In [172]:
!pip install pandas_profiling

     |████████████████████████████████| 133kB 4.3MB/s eta 0:00:01
     |████████████████████████████████| 614kB 5.1MB/s eta 0:00:01


  Stored in directory: /Users/jmcneilkeller/Library/Caches/pip/wheels/45/10/b5/5565b02c204a3bb87490a22e0e4c382ea06337bb79fd74d58a
  Stored in directory: /Users/jmcneilkeller/Library/Caches/pip/wheels/43/07/ac/7c5a9d708d65247ac1f94066cf1db075540b85716c30255459
  Stored in directory: /Users/jmcneilkeller/Library/Caches/pip/wheels/b0/b2/96/2074eee7dbf7b7df69d004c9b6ac4e32dad04fb7666cf943bd
Successfully built pandas-profiling htmlmin confuse


In [173]:
import pandas_profiling

In [90]:
import sqlite3

In [25]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [4]:
ratings = pd.read_csv('ml-latest/ratings.csv')

In [80]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,307,3.5,1256677221
1,1,481,3.5,1256677456
2,1,1091,1.5,1256677471
3,1,1257,4.5,1256677460
4,1,1449,4.5,1256677264


In [5]:
movies = pd.read_csv('ml-latest/movies.csv')

In [30]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58098 entries, 0 to 58097
Data columns (total 3 columns):
movieId    58098 non-null int64
title      58098 non-null object
genres     58098 non-null object
dtypes: int64(1), object(2)
memory usage: 1.3+ MB


In [6]:
tags = pd.read_csv('ml-latest/tags.csv')

In [29]:
tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1108997 entries, 0 to 1108996
Data columns (total 4 columns):
userId       1108997 non-null int64
movieId      1108997 non-null int64
tag          1108981 non-null object
timestamp    1108997 non-null int64
dtypes: int64(3), object(1)
memory usage: 33.8+ MB


In [7]:
g_tags = pd.read_csv('ml-latest/genome-tags.csv')

In [42]:
g_tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1128 entries, 0 to 1127
Data columns (total 2 columns):
tagId    1128 non-null int64
tag      1128 non-null object
dtypes: int64(1), object(1)
memory usage: 17.7+ KB


In [8]:
links = pd.read_csv('ml-latest/links.csv')

In [28]:
links.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58098 entries, 0 to 58097
Data columns (total 3 columns):
movieId    58098 non-null int64
imdbId     58098 non-null int64
tmdbId     57917 non-null float64
dtypes: float64(1), int64(2)
memory usage: 1.3 MB


In [9]:
g_scores = pd.read_csv('ml-latest/genome-scores.csv')

In [26]:
g_scores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14862528 entries, 0 to 14862527
Data columns (total 3 columns):
movieId      int64
tagId        int64
relevance    float64
dtypes: float64(1), int64(2)
memory usage: 340.2 MB


In [21]:
movies.movieId[37258]

146485

In [47]:
links.imdbId.max()

9038520

In [39]:
df_master = movies.merge(links,on='movieId')
df_master.head()

,movieId,title,genres,imdbId,tmdbId
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0
1,2,Jumanji (1995),Adventure|Children|Fantasy,113497,8844.0
2,3,Grumpier Old Men (1995),Comedy|Romance,113228,15602.0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,114885,31357.0
4,5,Father of the Bride Part II (1995),Comedy,113041,11862.0


In [62]:
df_master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58098 entries, 0 to 58097
Data columns (total 5 columns):
movieId    58098 non-null int64
title      58098 non-null object
genres     58098 non-null object
imdbId     58098 non-null object
tmdbId     57917 non-null float64
dtypes: float64(1), int64(1), object(3)
memory usage: 2.7+ MB


In [57]:
def imdb_correcter(i):
    if len(str(i)) > 0:
        return 'tt' + '0'*(7-len(str(i))) + str(i)

In [60]:
# Updating 
df_master['imdbId'] = df_master.imdbId.apply(imdb_correcter)

In [61]:
df_master['imdbId']

0        tt0114709
1        tt0113497
2        tt0113228
3        tt0114885
4        tt0113041
5        tt0113277
6        tt0114319
7        tt0112302
8        tt0114576
9        tt0113189
10       tt0112346
11       tt0112896
12       tt0112453
13       tt0113987
14       tt0112760
15       tt0112641
16       tt0114388
17       tt0113101
18       tt0112281
19       tt0113845
20       tt0113161
21       tt0112722
22       tt0112401
23       tt0114168
24       tt0113627
25       tt0114057
26       tt0114011
27       tt0114117
28       tt0112682
29       tt0115012
30       tt0112792
31       tt0114746
32       tt0114952
33       tt0112431
34       tt0112637
35       tt0112818
36       tt0112286
37       tt0113442
38       tt0112697
39       tt0112749
40       tt0114279
41       tt0112819
42       tt0114272
43       tt0113855
44       tt0114681
45       tt0113347
46       tt0114369
47       tt0114148
48       tt0114916
49       tt0114814
50       tt0109950
51       tt0113819
52       tt0

In [63]:
df_master['tmdbId'] = df_master['tmdbId'].astype('object')
df_master['tmdbId']

0           862
1          8844
2         15602
3         31357
4         11862
5           949
6         11860
7         45325
8          9091
9           710
10         9087
11        12110
12        21032
13        10858
14         1408
15          524
16         4584
17            5
18         9273
19        11517
20         8012
21         1710
22         9691
23        12665
24          451
25        16420
26         9263
27        17015
28          902
29        37557
30         9909
31           63
32        78802
33         9598
34        47018
35          687
36       139405
37        33689
38         9603
39        34615
40        31174
41        11443
42        35196
43         9312
44          577
45        11861
46          807
47        10530
48         8391
49          629
50       117164
51        11448
52        49133
53        26441
54        97406
55       124057
56         9089
57        11010
58        99040
59        11359
60        17182
61         2054
62      

In [64]:
df_master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58098 entries, 0 to 58097
Data columns (total 5 columns):
movieId    58098 non-null int64
title      58098 non-null object
genres     58098 non-null object
imdbId     58098 non-null object
tmdbId     57917 non-null object
dtypes: int64(1), object(4)
memory usage: 2.7+ MB


In [70]:
df_master.loc[df_master.movieId == 1]

,movieId,title,genres,imdbId,tmdbId
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,tt0114709,862


In [71]:
g_scores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14862528 entries, 0 to 14862527
Data columns (total 3 columns):
movieId      int64
tagId        int64
relevance    float64
dtypes: float64(1), int64(2)
memory usage: 340.2 MB


In [67]:
g_tags

,tagId,tag
0,1,007
1,2,007 (series)
2,3,18th century
3,4,1920s
4,5,1930s
5,6,1950s
6,7,1960s
7,8,1970s
8,9,1980s
9,10,19th century


In [78]:
g_scores.head(100)

,movieId,tagId,relevance
0,1,1,0.02900
1,1,2,0.02375
2,1,3,0.05425
3,1,4,0.06875
4,1,5,0.16000
5,1,6,0.19525
6,1,7,0.07600
7,1,8,0.25200
8,1,9,0.22750
9,1,10,0.02400


In [91]:
conn = sqlite3.connect('goodbad.db')
cur = conn.cursor()

In [92]:
ratings.to_sql('ratings', conn, if_exists='append', index=False)

In [97]:
g_tags.to_sql('genome_tags', conn, if_exists='append', index=False)

In [99]:
g_scores.to_sql('genome_scores', conn, if_exists='append', index=False)

In [100]:
movies.to_sql('movies', conn, if_exists='append', index=False)

In [101]:
tags.to_sql('tags', conn, if_exists='append', index=False)

In [102]:
links.to_sql('links', conn, if_exists='append', index=False)

In [146]:
ratings.movieId.unique()

array([   307,    481,   1091, ..., 117857, 133409, 142855])

In [150]:
ratings

,userId,movieId,rating,timestamp
0,1,307,3.5,1256677221
1,1,481,3.5,1256677456
2,1,1091,1.5,1256677471
3,1,1257,4.5,1256677460
4,1,1449,4.5,1256677264
5,1,1590,2.5,1256677236
6,1,1591,1.5,1256677475
7,1,2134,4.5,1256677464
8,1,2478,4.0,1256677239
9,1,2840,3.0,1256677500


In [152]:
ratings.rename(columns={'userId':'count'},inplace=True)

In [154]:
ratings.drop(['timestamp'],inplace=True,axis=1)

In [155]:
ratbin = ratings.groupby(['movieId','rating']).count()

In [156]:
ratbin.head()

count
movieId rating       
1       0.5       478
        1.0       840
        1.5       468
        2.0      2160
        2.5      1665

In [157]:
unstack = ratbin.unstack()
unstack.head()

count                                                          \
rating     0.5    1.0    1.5     2.0     2.5      3.0     3.5      4.0   
movieId                                                                  
1        478.0  840.0  468.0  2160.0  1665.0  11872.0  6095.0  22710.0   
2        276.0  813.0  542.0  2551.0  1902.0   8711.0  3116.0   6303.0   
3        174.0  777.0  237.0  1677.0   629.0   6227.0   774.0   3440.0   
4         48.0  352.0   31.0   497.0    77.0   1136.0    68.0    557.0   
5        248.0  802.0  282.0  1737.0   681.0   6655.0   761.0   2989.0   

                          
rating      4.5      5.0  
movieId                   
1        5684.0  16497.0  
2         790.0   2139.0  
3         159.0   1491.0  
4          12.0    211.0  
5         174.0   1145.0

In [158]:
unstack.isnull().sum()

       rating
count  0.5       30889
       1.0       33213
       1.5       34230
       2.0       27150
       2.5       23590
       3.0       16780
       3.5       19136
       4.0       21634
       4.5       30302
       5.0       29680
dtype: int64

In [159]:
df_master = df_master.merge(unstack,on='movieId')

In [160]:
df_master.isnull().sum()

movieId             0
title               0
genres              0
imdbId              0
tmdbId            177
(count, 0.5)    30889
(count, 1.0)    33213
(count, 1.5)    34230
(count, 2.0)    27150
(count, 2.5)    23590
(count, 3.0)    16780
(count, 3.5)    19136
(count, 4.0)    21634
(count, 4.5)    30302
(count, 5.0)    29680
dtype: int64

In [161]:
df_master.head(10)

,movieId,title,genres,imdbId,tmdbId,"(count, 0.5)","(count, 1.0)","(count, 1.5)","(count, 2.0)","(count, 2.5)","(count, 3.0)","(count, 3.5)","(count, 4.0)","(count, 4.5)","(count, 5.0)"
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,tt0114709,862,478.0,840.0,468.0,2160.0,1665.0,11872.0,6095.0,22710.0,5684.0,16497.0
1,2,Jumanji (1995),Adventure|Children|Fantasy,tt0113497,8844,276.0,813.0,542.0,2551.0,1902.0,8711.0,3116.0,6303.0,790.0,2139.0
2,3,Grumpier Old Men (1995),Comedy|Romance,tt0113228,15602,174.0,777.0,237.0,1677.0,629.0,6227.0,774.0,3440.0,159.0,1491.0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,tt0114885,31357,48.0,352.0,31.0,497.0,77.0,1136.0,68.0,557.0,12.0,211.0
4,5,Father of the Bride Part II (1995),Comedy,tt0113041,11862,248.0,802.0,282.0,1737.0,681.0,6655.0,761.0,2989.0,174.0,1145.0
5,6,Heat (1995),Action|Crime|Thriller,tt0113277,949,53.0,339.0,71.0,1013.0,415.0,6740.0,1994.0,9613.0,2296.0,6149.0
6,7,Sabrina (1995),Comedy|Romance,tt0114319,11860,141.0,409.0,155.0,1186.0,489.0,5978.0,748.0,4122.0,253.0,1820.0
7,8,Tom and Huck (1995),Adventure|Children,tt0112302,45325,14.0,87.0,13.0,181.0,55.0,669.0,60.0,318.0,13.0,129.0
8,9,Sudden Death (1995),Action,tt0114576,9091,25.0,310.0,59.0,587.0,148.0,2089.0,90.0,853.0,17.0,271.0
9,10,GoldenEye (1995),Action|Adventure|Thriller,tt0113189,710,166.0,632.0,226.0,2038.0,1058.0,12189.0,2882.0,9909.0,912.0,3074.0


In [167]:
tags = [1058,933,934,891,616,274,275,276,198]
g_rev = g_scores.loc[g_scores['tagId'].isin(tags)]
g_rev.head()


,movieId,tagId,relevance
197,1,198,0.07500
273,1,274,0.21400
274,1,275,0.30725
275,1,276,0.21925
615,1,616,0.07300


In [ ]:
g_rev.pivot_table(index=

In [168]:
g_rev.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 118584 entries, 197 to 14862457
Data columns (total 3 columns):
movieId      118584 non-null int64
tagId        118584 non-null int64
relevance    118584 non-null float64
dtypes: float64(1), int64(2)
memory usage: 3.6 MB


In [180]:
g_test = g_rev.pivot(index='movieId',columns='tagId',values='relevance')
g_test.head(100)

tagId,198,274,275,276,616,891,933,934,1058
movieId,,,,,,,,,
1,0.07500,0.21400,0.30725,0.21925,0.07300,0.11750,0.13650,0.01150,0.01875
2,0.19625,0.11075,0.11025,0.19675,0.04200,0.04500,0.28000,0.02125,0.04200
3,0.06900,0.04100,0.10225,0.17550,0.02775,0.04600,0.22900,0.03375,0.03425
4,0.07775,0.04275,0.10675,0.21600,0.05475,0.02800,0.28075,0.01600,0.03725
5,0.08750,0.02750,0.08375,0.17825,0.02875,0.02925,0.24200,0.01750,0.03850
6,0.02825,0.15575,0.36400,0.40450,0.06575,0.04675,0.15225,0.01675,0.02250
7,0.09875,0.02550,0.11000,0.14500,0.01825,0.03275,0.23550,0.03175,0.04775
8,0.06325,0.02575,0.07950,0.19175,0.02675,0.02100,0.26150,0.01950,0.02725
9,0.17800,0.02500,0.11650,0.18300,0.03300,0.02550,0.50225,0.05100,0.07700


In [181]:
g_tag_test = df_master.merge(g_test, on='movieId', )
g_tag_test.head()

,movieId,title,genres,imdbId,tmdbId,"(count, 0.5)","(count, 1.0)","(count, 1.5)","(count, 2.0)","(count, 2.5)","(count, 3.0)","(count, 3.5)","(count, 4.0)","(count, 4.5)","(count, 5.0)",198,274,275,276,616,891,933,934,1058
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,tt0114709,862,478.0,840.0,468.0,2160.0,1665.0,11872.0,6095.0,22710.0,5684.0,16497.0,0.07500,0.21400,0.30725,0.21925,0.07300,0.11750,0.13650,0.01150,0.01875
1,2,Jumanji (1995),Adventure|Children|Fantasy,tt0113497,8844,276.0,813.0,542.0,2551.0,1902.0,8711.0,3116.0,6303.0,790.0,2139.0,0.19625,0.11075,0.11025,0.19675,0.04200,0.04500,0.28000,0.02125,0.04200
2,3,Grumpier Old Men (1995),Comedy|Romance,tt0113228,15602,174.0,777.0,237.0,1677.0,629.0,6227.0,774.0,3440.0,159.0,1491.0,0.06900,0.04100,0.10225,0.17550,0.02775,0.04600,0.22900,0.03375,0.03425
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,tt0114885,31357,48.0,352.0,31.0,497.0,77.0,1136.0,68.0,557.0,12.0,211.0,0.07775,0.04275,0.10675,0.21600,0.05475,0.02800,0.28075,0.01600,0.03725
4,5,Father of the Bride Part II (1995),Comedy,tt0113041,11862,248.0,802.0,282.0,1737.0,681.0,6655.0,761.0,2989.0,174.0,1145.0,0.08750,0.02750,0.08375,0.17825,0.02875,0.02925,0.24200,0.01750,0.03850


In [182]:
g_tag_test.to_csv('gtag_test.csv')

In [184]:
g_tag_test.describe()

,movieId,"(count, 0.5)","(count, 1.0)","(count, 1.5)","(count, 2.0)","(count, 2.5)","(count, 3.0)","(count, 3.5)","(count, 4.0)","(count, 4.5)","(count, 5.0)",198,274,275,276,616,891,933,934,1058
count,13176.000000,12440.000000,12311.000000,11877.000000,13025.000000,13077.000000,13174.000000,13127.000000,13166.000000,12686.000000,12962.000000,13176.000000,13176.000000,13176.000000,13176.00000,13176.000000,13176.000000,13176.000000,13176.000000,13176.000000
mean,39970.868549,34.110450,70.896434,36.210996,140.096430,102.356657,413.408911,254.531195,557.373234,185.417626,312.359667,0.092009,0.125118,0.246457,0.28432,0.127759,0.059628,0.291982,0.035529,0.037644
std,48151.129249,80.622654,182.791444,82.185694,338.208791,226.199578,1113.128833,621.079637,1649.334703,674.527451,1437.243790,0.106802,0.133213,0.200088,0.15842,0.151946,0.075956,0.164562,0.058301,0.055112
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.007250,0.006250,0.018000,0.06175,0.007000,0.004750,0.051000,0.003000,0.005250
25%,3680.750000,3.000000,4.000000,3.000000,8.000000,8.000000,22.000000,18.000000,22.000000,6.000000,8.000000,0.030000,0.041000,0.110750,0.17925,0.041750,0.028500,0.169750,0.011750,0.014000
50%,7880.000000,7.000000,12.000000,8.000000,24.000000,21.000000,66.000000,45.000000,69.000000,20.000000,25.000000,0.052125,0.075500,0.171750,0.23475,0.072625,0.039000,0.239250,0.018250,0.020000
75%,71249.000000,25.000000,54.000000,28.000000,107.000000,78.000000,277.750000,169.000000,308.000000,86.000000,115.000000,0.107000,0.155750,0.306875,0.33400,0.142250,0.057500,0.372250,0.034500,0.036500
max,187595.000000,1132.000000,4339.000000,1219.000000,5687.000000,3016.000000,22333.000000,7373.000000,28340.000000,14110.000000,48762.000000,0.847000,0.968000,0.994500,0.99325,0.988250,1.000000,0.952500,0.897500,0.806750


In [187]:
g_tag_test['total_count'] = g_tag_test.iloc[:,5:15].sum(axis=1)
g_tag_test.head()

,movieId,title,genres,imdbId,tmdbId,"(count, 0.5)","(count, 1.0)","(count, 1.5)","(count, 2.0)","(count, 2.5)","(count, 3.0)","(count, 3.5)","(count, 4.0)","(count, 4.5)","(count, 5.0)",198,274,275,276,616,891,933,934,1058,total_count
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,tt0114709,862,478.0,840.0,468.0,2160.0,1665.0,11872.0,6095.0,22710.0,5684.0,16497.0,0.07500,0.21400,0.30725,0.21925,0.07300,0.11750,0.13650,0.01150,0.01875,68469.0
1,2,Jumanji (1995),Adventure|Children|Fantasy,tt0113497,8844,276.0,813.0,542.0,2551.0,1902.0,8711.0,3116.0,6303.0,790.0,2139.0,0.19625,0.11075,0.11025,0.19675,0.04200,0.04500,0.28000,0.02125,0.04200,27143.0
2,3,Grumpier Old Men (1995),Comedy|Romance,tt0113228,15602,174.0,777.0,237.0,1677.0,629.0,6227.0,774.0,3440.0,159.0,1491.0,0.06900,0.04100,0.10225,0.17550,0.02775,0.04600,0.22900,0.03375,0.03425,15585.0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,tt0114885,31357,48.0,352.0,31.0,497.0,77.0,1136.0,68.0,557.0,12.0,211.0,0.07775,0.04275,0.10675,0.21600,0.05475,0.02800,0.28075,0.01600,0.03725,2989.0
4,5,Father of the Bride Part II (1995),Comedy,tt0113041,11862,248.0,802.0,282.0,1737.0,681.0,6655.0,761.0,2989.0,174.0,1145.0,0.08750,0.02750,0.08375,0.17825,0.02875,0.02925,0.24200,0.01750,0.03850,15474.0


In [188]:
g_tag_test.describe()

,movieId,"(count, 0.5)","(count, 1.0)","(count, 1.5)","(count, 2.0)","(count, 2.5)","(count, 3.0)","(count, 3.5)","(count, 4.0)","(count, 4.5)","(count, 5.0)",198,274,275,276,616,891,933,934,1058,total_count
count,13176.000000,12440.000000,12311.000000,11877.000000,13025.000000,13077.000000,13174.000000,13127.000000,13166.000000,12686.000000,12962.000000,13176.000000,13176.000000,13176.000000,13176.00000,13176.000000,13176.000000,13176.000000,13176.000000,13176.000000,13176.000000
mean,39970.868549,34.110450,70.896434,36.210996,140.096430,102.356657,413.408911,254.531195,557.373234,185.417626,312.359667,0.092009,0.125118,0.246457,0.28432,0.127759,0.059628,0.291982,0.035529,0.037644,2080.856254
std,48151.129249,80.622654,182.791444,82.185694,338.208791,226.199578,1113.128833,621.079637,1649.334703,674.527451,1437.243790,0.106802,0.133213,0.200088,0.15842,0.151946,0.075956,0.164562,0.058301,0.055112,5655.243811
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.007250,0.006250,0.018000,0.06175,0.007000,0.004750,0.051000,0.003000,0.005250,48.000000
25%,3680.750000,3.000000,4.000000,3.000000,8.000000,8.000000,22.000000,18.000000,22.000000,6.000000,8.000000,0.030000,0.041000,0.110750,0.17925,0.041750,0.028500,0.169750,0.011750,0.014000,118.000000
50%,7880.000000,7.000000,12.000000,8.000000,24.000000,21.000000,66.000000,45.000000,69.000000,20.000000,25.000000,0.052125,0.075500,0.171750,0.23475,0.072625,0.039000,0.239250,0.018250,0.020000,345.000000
75%,71249.000000,25.000000,54.000000,28.000000,107.000000,78.000000,277.750000,169.000000,308.000000,86.000000,115.000000,0.107000,0.155750,0.306875,0.33400,0.142250,0.057500,0.372250,0.034500,0.036500,1354.000000
max,187595.000000,1132.000000,4339.000000,1219.000000,5687.000000,3016.000000,22333.000000,7373.000000,28340.000000,14110.000000,48762.000000,0.847000,0.968000,0.994500,0.99325,0.988250,1.000000,0.952500,0.897500,0.806750,97999.000000


In [174]:
pandas_profiling.ProfileReport(g_rev)